In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install fsspec==2023.9.2
    !pip install -U transformers trl
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from datasets import load_dataset
#dataset = load_dataset("deebak14/rhinoscript_ft_data_04", split = "train")

train_dataset = load_dataset("deebak14/rhinoscript_ft_data_04", split = "train")
eval_dataset = load_dataset("deebak14/rhinoscript_ft_data_04_test", split = "train")


README.md:   0%|          | 0.00/349 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.21M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9262 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/346 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/446k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1364 [00:00<?, ? examples/s]

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = "Qwen/Qwen3-14B"            #"Qwen/Qwen2.5-Coder-7B-Instruct" #"mistralai/Mistral-7B-Instruct-v0.3" #"google/gemma-3-12b-it"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Setup bitsandbytes 4-bit config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# Load the model in 4bit
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",         # auto-assigns model to GPU if available
    torch_dtype=torch.float16, # or 'auto' if you want auto-detection
)

# Set max sequence length when tokenizing/generating, not in .from_pretrained
max_seq_length = 4096


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

...

model-00003-of-00008.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/3.84G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/1.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [5]:
from peft import LoraConfig, get_peft_model

# Configure LoRA
lora_config = LoraConfig(
    r=4,                     # LoRA rank
    lora_alpha=8,           # LoRA alpha (often 2x r)
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.01,
    bias="none",
    task_type="CAUSAL_LM",    # Important for language modeling
)

# Attach LoRA to model
model = get_peft_model(model, lora_config)

# (Optional) Print trainable parameters to confirm LoRA is applied
model.print_trainable_parameters()

trainable params: 16,056,320 || all params: 14,784,363,520 || trainable%: 0.1086


In [6]:
with open("/workspace/qwen_chat_template.jinja") as f:
    tokenizer.chat_template = f.read()

In [7]:
from trl import apply_chat_template

# Pick a few examples from your dataset
for i in range(3):
    sample = train_dataset[i]
    # Apply the chat template with tokenization and assistant mask
    output = tokenizer.apply_chat_template(
        sample['messages'],
        tokenize=True,
        return_assistant_tokens_mask=True,
        return_dict=True,
    )

    print(f"Sample {i}:")
    print("".join(map(str, output["assistant_masks"])))
    print("-" * 40)

Sample 0:
000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000011111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111100
----------------------------------------
Sample 1:
0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111100
----------------------------------------
Sample 2:
00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000111111111111111111111111111111111111111111111111111111111111111111111100
---------------------------------------

In [9]:
from trl import SFTTrainer, SFTConfig

sft_config = SFTConfig(
    dataset_text_field="messages",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_ratio=0.05,                # Increased for more stable start
    num_train_epochs=5,
    learning_rate= 2e-6,             #1e-5             # Lowered for better generalization
    logging_steps=25,
    optim="adamw_8bit",
    weight_decay=0.05,
    lr_scheduler_type="cosine",
    seed=3407,
    report_to="none",
    eval_strategy="epoch",          # Evaluate and save every epoch
    save_strategy="epoch",
    save_total_limit=2,             # Only keep last 3 checkpoints
    max_grad_norm=1.0,              # Clip gradients
    bf16=True,  # set this instead of fp16=True
    fp16=False,
    push_to_hub=False,
    neftune_noise_alpha=5,
    assistant_only_loss=True,
    chat_template_path="/workspace/qwen_chat_template.jinja"
)

trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)


Tokenizing train dataset:   0%|          | 0/9262 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/9262 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/1364 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/1364 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [10]:
# Get the full input_ids and assistant_masks for a sample
sample = trainer.train_dataset[100]
input_ids = sample["input_ids"]
assistant_masks = sample["assistant_masks"]

# Extract token ids for assistant tokens only
assistant_token_ids = [tid for tid, mask in zip(input_ids, assistant_masks) if mask == 1]

# Now decode ONLY those assistant tokens
assistant_text = tokenizer.decode(assistant_token_ids)
print("Assistant text only:\n", assistant_text)


Assistant text only:
 <|im_start|>assistant
<think>

</think>

To determine if a surface is uniform in both directions using `SurfaceKnots`, check if the knot vectors have uniform spacing. Here's a script example:

```python
import rhinoscriptsyntax as rs

# Function to check uniformity
def is_uniform(knots):
    differences = [knots[i+1] - knots[i] for i in range(len(knots)-1)]
    return all(d == differences[0] for d in differences)

# Select a surface
surface_id = rs.GetObject("Select a surface", rs.filter.surface)

if rs.IsSurface(surface_id):
    knots = rs.SurfaceKnots(surface_id)
    if knots:
        u_knots, v_knots = knots
        uniform_u = is_uniform(u_knots)
        uniform_v = is_uniform(v_knots)
        print("Uniform in U direction:", uniform_u)
        print("Uniform in V direction:", uniform_v)
else:
    print("The selected object is not a surface.")
```



In [12]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA H100 PCIe. Max memory = 79.097 GB.
13.855 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [13]:
trainer_stats = trainer.train()

Epoch,Training Loss,Validation Loss
1,0.587100,0.940676
2,0.500800,0.866793
3,0.498500,0.838929
4,0.471600,0.829141
5,0.474600,0.827468


In [11]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

3875.2585 seconds used for training.
64.59 minutes used for training.
Peak reserved memory = 32.387 GB.
Peak reserved memory for training = 18.532 GB.
Peak reserved memory % of max memory = 81.874 %.
Peak reserved memory for training % of max memory = 46.849 %.


<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Qwen-3` team, the recommended settings for reasoning inference are `temperature = 0.6, top_p = 0.95, top_k = 20`

For normal chat based inference, `temperature = 0.7, top_p = 0.8, top_k = 20`

# After SFT

In [ ]:
from transformers import pipeline

# Use a pipeline for easy inference
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Example usage
messages = [
    {"role" : "system",
     "content" : """You are an expert in Python and scripting using the rhinoscriptsyntax module. Your task is to understand user query and generate a script accordingly to model in rhino 3d.
     Please reason step by step. Breakdown the user query into parts, based on which write an algorithm, then write the code. Do not prompt user for input in the code unless specified. user pythons math module for math related tasks
     Your Output should be a python code inside ```python ``` block. """},
    {"role" : "user",
     "content" : "create 8 cubes along the circle"}
]

prompt = pipe.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking = False
)

outputs = pipe(
        prompt,
        max_new_tokens=4096,
        do_sample=True,
        temperature=0.7,
        top_p=0.8,
        top_k=20,
        min_p=0
)
print(outputs[0]["generated_text"])

Device set to use cuda:0
